# Understanding Large Language Models: A Step-by-Step Journey

## Our Goal

In this notebook, we will understand how language models predict text by following the complete process of predicting "we love deep learning" word by word. We'll explore four fundamental concepts that make modern language models like ChatGPT work:

1. **Forward Pass**: How models generate predictions
2. **Loss Calculation**: How we measure prediction quality
3. **Backpropagation**: How we identify what needs improvement
4. **Gradient Descent**: How we make those improvements

## Our Vocabulary and Target

We'll work with a simple vocabulary to keep things clear and manageable. Our model will learn to predict each word in our target sequence step by step.


In [1]:
# Setup our simple vocabulary and target sequence
VOCAB = ["<BOS>", "we", "love", "deep", "learning", "<EOS>", "the", "is", "great", "model", "hello", "world"]
target_sequence = ["we", "love", "deep", "learning"]

print("VOCABULARY:", VOCAB)
print("TARGET SEQUENCE:", target_sequence)
print("VOCABULARY SIZE:", len(VOCAB))

# Initialize simple model parameters (weights) - these will be updated during training
# In real models, these would be millions or billions of parameters
model_parameters = {
    'layer1_weights': [0.1, -0.3, 0.5, 0.2, -0.1, 0.4, 0.8, -0.2, 0.3, -0.5, 0.7, -0.4],
    'layer2_weights': [0.2, 0.1, -0.4, 0.6, 0.3, -0.2, -0.1, 0.5, -0.3, 0.4, -0.6, 0.1],
    'output_weights': [0.3, -0.1, 0.4, -0.2, 0.5, 0.1, -0.3, 0.2, 0.6, -0.4, 0.1, 0.3]
}

print("\nInitial model parameters (simplified representation):")
print("Layer 1 weights:", len(model_parameters['layer1_weights']), "parameters")
print("Layer 2 weights:", len(model_parameters['layer2_weights']), "parameters") 
print("Output weights:", len(model_parameters['output_weights']), "parameters")

VOCABULARY: ['<BOS>', 'we', 'love', 'deep', 'learning', '<EOS>', 'the', 'is', 'great', 'model', 'hello', 'world']
TARGET SEQUENCE: ['we', 'love', 'deep', 'learning']
VOCABULARY SIZE: 12

Initial model parameters (simplified representation):
Layer 1 weights: 12 parameters
Layer 2 weights: 12 parameters
Output weights: 12 parameters


## What Are Logits?

Logits are the raw numerical scores that a language model assigns to every word in its vocabulary when predicting the next word. Think of logits as the model's initial "gut feeling" about how likely each word is to come next, before any normalization.

**Key Properties of Logits:**

- They can be any real number (positive, negative, large, small)
- Higher logits indicate the model thinks a word is more likely
- Lower logits indicate the model thinks a word is less likely
- They are computed by passing the current context through the neural network layers

Let's see what logits look like when our model tries to predict the first word after the beginning-of-sequence token.


In [3]:
def demonstrate_logits():
    # Pre-calculated realistic logit values for predicting first word after <BOS>
    logits_after_bos = {
        "we": 2.1,      # Target word - decent score
        "the": 3.2,     # Highest - most common starter
        "hello": 2.8,   # High - common greeting
        "is": 1.5,      # Medium - possible starter
        "love": -0.5,   # Low - uncommon starter
        "deep": -1.2,   # Lower - rare starter
        "learning": -2.0, # Very low - very rare starter
        "<EOS>": -10.0, # Impossible - can't start with end token
    }
    
    print("Logit scores for predicting first word after <BOS>:")
    print("=" * 50)
    for word, logit in sorted(logits_after_bos.items(), key=lambda x: x[1], reverse=True):
        marker = " <- Our target!" if word == "we" else ""
        print(f"{word:10}: {logit:6.1f}{marker}")
    
    print(f"\nNotice how 'the' has the highest logit ({logits_after_bos['the']}) because it's")
    print("the most common way to start sentences in English.")
    print(f"Our target word 'we' has a logit of {logits_after_bos['we']}, which is decent")
    print("but not the highest - the model will need training to improve this!")
    
    return logits_after_bos

# Run the demonstration
logits_step1 = demonstrate_logits()

Logit scores for predicting first word after <BOS>:
the       :    3.2
hello     :    2.8
we        :    2.1 <- Our target!
is        :    1.5
love      :   -0.5
deep      :   -1.2
learning  :   -2.0
<EOS>     :  -10.0

Notice how 'the' has the highest logit (3.2) because it's
the most common way to start sentences in English.
Our target word 'we' has a logit of 2.1, which is decent
but not the highest - the model will need training to improve this!


## From Logits to Probabilities: The Softmax Operation

Raw logits are useful for the model internally, but they're not easy for us to interpret. We need to convert them into probabilities - numbers between 0 and 1 that sum to exactly 1.0. This conversion is done using the **softmax function**.

**The Softmax Process:**

1. Calculate e^(logit) for each word (this makes all values positive)
2. Sum all these exponential values
3. Divide each exponential value by the sum

This ensures we get valid probabilities that represent the model's confidence in each possible next word.


In [4]:
import math

def demonstrate_softmax(logits_dict):
    print("SOFTMAX CONVERSION: From Logits to Probabilities")
    print("=" * 55)
    
    words = list(logits_dict.keys())
    logits = list(logits_dict.values())
    
    print("STEP 1: Calculate e^(logit) for each word")
    print("-" * 40)
    exp_logits = []
    for word, logit in zip(words, logits):
        exp_val = math.exp(logit)
        exp_logits.append(exp_val)
        print(f"e^({logit:5.1f}) = {exp_val:8.2f}  for '{word}'")
    
    print(f"\nSTEP 2: Sum all exponential values")
    print("-" * 35)
    total = sum(exp_logits)
    print(f"Total sum = {total:.2f}")
    
    print(f"\nSTEP 3: Calculate final probabilities")
    print("-" * 38)
    probabilities = {}
    for word, exp_val in zip(words, exp_logits):
        prob = exp_val / total
        probabilities[word] = prob
        marker = " <- Our target!" if word == "we" else ""
        print(f"P({word:8}) = {exp_val:8.2f} / {total:.2f} = {prob:.4f}{marker}")
    
    print(f"\nVerification: All probabilities sum to {sum(probabilities.values()):.6f}")
    print(f"\nKey insight: '{max(logits_dict, key=logits_dict.get)}' had the highest logit")
    print(f"and now has the highest probability ({max(probabilities.values()):.4f})")
    
    return probabilities

# Convert our logits to probabilities
probabilities_step1 = demonstrate_softmax(logits_step1)

SOFTMAX CONVERSION: From Logits to Probabilities
STEP 1: Calculate e^(logit) for each word
----------------------------------------
e^(  2.1) =     8.17  for 'we'
e^(  3.2) =    24.53  for 'the'
e^(  2.8) =    16.44  for 'hello'
e^(  1.5) =     4.48  for 'is'
e^( -0.5) =     0.61  for 'love'
e^( -1.2) =     0.30  for 'deep'
e^( -2.0) =     0.14  for 'learning'
e^(-10.0) =     0.00  for '<EOS>'

STEP 2: Sum all exponential values
-----------------------------------
Total sum = 54.67

STEP 3: Calculate final probabilities
--------------------------------------
P(we      ) =     8.17 / 54.67 = 0.1494 <- Our target!
P(the     ) =    24.53 / 54.67 = 0.4488
P(hello   ) =    16.44 / 54.67 = 0.3008
P(is      ) =     4.48 / 54.67 = 0.0820
P(love    ) =     0.61 / 54.67 = 0.0111
P(deep    ) =     0.30 / 54.67 = 0.0055
P(learning) =     0.14 / 54.67 = 0.0025
P(<EOS>   ) =     0.00 / 54.67 = 0.0000

Verification: All probabilities sum to 1.000000

Key insight: 'the' had the highest logit
and now h

## Sequential Prediction: How Context Shapes Predictions

Language models don't just predict single words in isolation - they use the entire preceding context to make increasingly informed predictions. As we build up the sequence "we love deep learning," each new word provides more context that helps the model make better predictions for the next word.

Let's observe how the model's logit scores change as we provide more context at each step.

In [2]:
def demonstrate_sequence_prediction():
    print("COMPLETE SEQUENCE PREDICTION: 'we love deep learning'")
    print("=" * 60)
    
    # Pre-calculated logits for each prediction step showing how context improves predictions
    prediction_steps = [
        {
            "step": 1,
            "context": "<BOS>",
            "target": "we", 
            "logits": {"we": 2.1, "love": -0.5, "deep": -1.2, "learning": -2.0, "the": 3.2, "hello": 2.8, "is": 1.5, "<EOS>": -10.0},
            "explanation": "'we' is a common sentence starter, though 'the' is even more common in general text"
        },
        {
            "step": 2,
            "context": "we",
            "target": "love",
            "logits": {"we": -3.0, "love": 2.8, "deep": -0.5, "learning": -1.5, "the": 1.2, "hello": -5.0, "is": 2.1, "<EOS>": -8.0},
            "explanation": "After 'we', action words like 'love', 'are', 'have' become much more likely than nouns"
        },
        {
            "step": 3,
            "context": "we love",
            "target": "deep",
            "logits": {"we": -5.0, "love": -4.0, "deep": 1.9, "learning": 0.8, "the": 0.5, "hello": -6.0, "is": -2.0, "<EOS>": -7.0},
            "explanation": "After 'we love', we expect objects or concepts; 'deep' scores well as it often precedes 'learning'"
        },
        {
            "step": 4,
            "context": "we love deep",
            "target": "learning",
            "logits": {"we": -6.0, "love": -5.0, "deep": -4.0, "learning": 3.5, "the": -2.0, "hello": -8.0, "is": -3.0, "<EOS>": -6.0},
            "explanation": "'deep learning' is a common collocation - 'learning' becomes very likely after 'deep' in this context"
        }
    ]
    
    all_step_probabilities = []
    
    for step_info in prediction_steps:
        print(f"\nSTEP {step_info['step']}: Context = '{step_info['context']}' -> Predicting '{step_info['target']}'")
        print("-" * 70)
        
        # Show top 5 logits
        sorted_logits = sorted(step_info['logits'].items(), key=lambda x: x[1], reverse=True)
        print("Top 5 logit scores:")
        for i, (word, logit) in enumerate(sorted_logits[:5]):
            marker = " ** TARGET **" if word == step_info['target'] else ""
            print(f"  {i+1}. {word:10}: {logit:6.1f}{marker}")
        
        # Calculate probability for target word
        target_logit = step_info['logits'][step_info['target']]
        exp_values = [math.exp(logit) for logit in step_info['logits'].values()]
        total_exp = sum(exp_values)
        target_prob = math.exp(target_logit) / total_exp
        all_step_probabilities.append(target_prob)
        
        print(f"\nTarget word '{step_info['target']}' probability: {target_prob:.3f}")
        print(f"Context effect: {step_info['explanation']}")
    
    return prediction_steps, all_step_probabilities

# Generate predictions for the complete sequence
steps_data, step_probabilities = demonstrate_sequence_prediction()

COMPLETE SEQUENCE PREDICTION: 'we love deep learning'

STEP 1: Context = '<BOS>' -> Predicting 'we'
----------------------------------------------------------------------
Top 5 logit scores:
  1. the       :    3.2
  2. hello     :    2.8
  3. we        :    2.1 ** TARGET **
  4. is        :    1.5
  5. love      :   -0.5

Target word 'we' probability: 0.149
Context effect: 'we' is a common sentence starter, though 'the' is even more common in general text

STEP 2: Context = 'we' -> Predicting 'love'
----------------------------------------------------------------------
Top 5 logit scores:
  1. love      :    2.8 ** TARGET **
  2. is        :    2.1
  3. the       :    1.2
  4. deep      :   -0.5
  5. learning  :   -1.5

Target word 'love' probability: 0.571
Context effect: After 'we', action words like 'love', 'are', 'have' become much more likely than nouns

STEP 3: Context = 'we love' -> Predicting 'deep'
----------------------------------------------------------------------
Top 5 l

## Loss Function: Quantifying Prediction Quality

The **loss function** is how we measure how well our model is performing. Specifically, it measures how "surprised" the model is when it sees the correct answer. The mathematical formulation we use is called **cross-entropy loss** or **log-likelihood loss**.

**Key Concepts:**
- Lower loss = better predictions = less surprise when seeing the correct answer
- Higher loss = worse predictions = more surprise when seeing the correct answer  
- Loss is calculated as: Loss = -log(probability of correct word)
- Training aims to minimize the total loss across all predictions

The logarithm has a **useful property**: it heavily penalizes very low probabilities. If the model assigns a probability of 0.01 to the correct word, the loss is much higher than if it assigns 0.1.

### The Training Dataset as Ground Truth

Before we dive into the mathematical details, it's crucial to understand what the loss function is actually measuring and why it works. The loss function compares our model's predictions against what we call the **"ground truth"** - the training dataset that contains the correct answers.

**Training Dataset as Golden Labels:**

When we train a language model, we provide it with a massive dataset of text (books, articles, websites, etc.). This dataset represents our best approximation of "correct" language use. Each sequence in this dataset serves as:

- **Ground Truth**: The actual words that should come next in real language
- **Golden Labels**: The target answers our model should learn to predict
- **Reference Distribution**: The pattern of language we want our model to internalize

For our example sequence "we love deep learning", this represents a small piece of ground truth data. When we calculate loss, we're measuring how well our model's predictions align with this established pattern of language.

**From Random Weights to Meaningful Predictions:**

Remember our initial model parameters from Section 1? Those weights were randomly initialized:

```python
model_parameters = {
    'layer1_weights': [0.1, -0.3, 0.5, 0.2, -0.1, 0.4, ...],
    'layer2_weights': [0.2, 0.1, -0.4, 0.6, 0.3, -0.2, ...],
    'output_weights': [0.3, -0.1, 0.4, -0.2, 0.5, 0.1, ...]
}
```

These random numbers produce the predictions we saw in previous sections. The model doesn't "know" anything about language initially - its probabilities for predicting "we" (0.149) or "love" (0.571) come purely from these random mathematical operations.

**The Training Objective: Fitting to the Dataset Distribution**

The fundamental goal of training is to **fit the model to the training dataset's distribution**. This means:

1. **Statistical Matching**: Adjust the weights so the model's predicted probabilities match the frequency patterns in the training data
2. **Contextual Learning**: Teach the model that certain word combinations (like "deep learning") are more common than others
3. **Pattern Internalization**: Embed the statistical regularities of human language into the neural network weights

When we minimize the cross-entropy loss across millions of training examples, we're essentially forcing the model to internalize the probability distribution of the training dataset. The model learns that "the" is more common after "<BOS>" because it appears frequently in training data, not because it has any inherent understanding of language.

**Inductive Bias: The Hidden Consequence of Training Data**

This training process creates what we call an **inductive bias** - a set of assumptions and tendencies that the model develops based on its training data. This bias manifests in several ways:

1. **Domain Bias**: If trained on academic papers, the model will favor formal language
2. **Cultural Bias**: Training data reflects the cultural context of its sources
3. **Temporal Bias**: Models reflect the time period when their training data was collected
4. **Statistical Bias**: Common patterns in training data become "preferred" by the model

Our model's tendency to predict "the" with high probability after "<BOS>" isn't based on understanding - it's an inductive bias learned from statistical patterns in training data where "the" frequently starts sentences.

**The Circular Nature of Language Modeling:**

There's a profound circularity here: we use human-written text to train models to predict human-like text. The model doesn't learn "language" in an abstract sense - it learns to mimic the specific patterns present in its training distribution. This is why the choice of training data is so critical and why different models can have vastly different behaviors based on their training datasets.

The loss function, therefore, isn't just measuring prediction accuracy - it's measuring how well the model has absorbed and can reproduce the inductive biases present in its training data.

### The Mathematical Foundation: Next Token Prediction Loss

Now let's dive deep into the precise mathematical formulation used in Large Language Models. The loss function we use is called **Negative Log-Likelihood (NLL)** for **Next Token Prediction (NTP)**.

#### The Complete NLL Formula

For **Next Token Prediction** in Large Language Models, the negative log-likelihood formula is:

**NLL = -∑ᵢ₌₁ⁿ log P(xᵢ | x₁, x₂, ..., xᵢ₋₁; θ)**

**Where:**
- **xᵢ** = the i-th token in the sequence (e.g., "we", "love", "deep", "learning")
- **x₁, x₂, ..., xᵢ₋₁** = all previous tokens providing context
- **θ** = model parameters (our weights from Section 1)
- **n** = sequence length (4 tokens in our example)

#### Breaking Down Each Component

**1. Per-Token Probability Computation:**

The probability of each token given its context is computed using softmax:

```
P(xᵢ | context) = softmax(logits)ₓᵢ = exp(zₓᵢ) / ∑ⱼ₌₁ᵛ exp(zⱼ)
```

**Where:**
- **zₓᵢ** = logit score for token xᵢ (like our 2.1 for "we")
- **V** = vocabulary size (12 in our case)
- **j** indexes over entire vocabulary

#### Autoregressive Nature

The key insight is that each prediction depends on **all previous tokens**:

- **Step 1**: P("we" | "<BOS>") - no previous content words
- **Step 2**: P("love" | "we") - uses "we" as context  
- **Step 3**: P("deep" | "we", "love") - uses both previous tokens
- **Step 4**: P("learning" | "we", "love", "deep") - uses all three previous tokens

This is why our predictions improved with more context in Section 4 - the model had more information to work with.

#### Connection to Perplexity

The loss is often reported as **perplexity**: 

```
Perplexity = exp(NLL/n)
```

Lower perplexity indicates better language modeling performance.

This mathematical foundation captures the core training objective of all Large Language Models: **maximize the probability of the correct next token given the context**. Every word prediction in ChatGPT, GPT-4, or any other LLM follows this exact mathematical framework.

In [6]:
def demonstrate_cross_entropy_formula():
    print("CROSS-ENTROPY LOSS FORMULA: Step-by-Step Breakdown")
    print("=" * 60)
    
    print("The cross-entropy loss formula is: Loss = -log(P(correct_word))")
    print("Let's break this down with a concrete example:\n")
    
    # Use a simple toy example
    print("TOY EXAMPLE: Predicting 'cat' from vocabulary ['dog', 'cat', 'bird']")
    print("-" * 65)
    
    # Toy probabilities for demonstration
    toy_probabilities = {
        "dog": 0.2,
        "cat": 0.6,   # correct word
        "bird": 0.2
    }
    
    print("Model predictions (probabilities):")
    for word, prob in toy_probabilities.items():
        marker = " <- CORRECT ANSWER" if word == "cat" else ""
        print(f"  P({word}) = {prob:.1f}{marker}")
    
    print(f"\nVerification: {sum(toy_probabilities.values()):.1f} (probabilities sum to 1.0)")
    
    print(f"\nSTEP-BY-STEP LOSS CALCULATION:")
    print("-" * 35)
    
    correct_word = "cat"
    correct_prob = toy_probabilities[correct_word]
    
    print(f"Step 1: Identify the correct word")
    print(f"        Correct word = '{correct_word}'")
    
    print(f"\nStep 2: Find its probability")
    print(f"        P({correct_word}) = {correct_prob}")
    
    print(f"\nStep 3: Take the natural logarithm")
    log_prob = math.log(correct_prob)
    print(f"        log(P({correct_word})) = log({correct_prob}) = {log_prob:.4f}")
    
    print(f"\nStep 4: Apply the negative sign")
    loss = -log_prob
    print(f"        Loss = -log(P({correct_word})) = -({log_prob:.4f}) = {loss:.4f}")
    
    print(f"\nFINAL RESULT: Loss = {loss:.4f}")
    
    # Show why we use negative log
    print(f"\n" + "="*50)
    print("WHY NEGATIVE LOGARITHM?")
    print("="*50)
    
    prob_examples = [0.9, 0.5, 0.1, 0.01]
    print("Probability -> log(prob) -> -log(prob) (Loss)")
    print("-" * 45)
    
    for prob in prob_examples:
        log_val = math.log(prob)
        loss_val = -log_val
        quality = "Excellent" if prob > 0.8 else "Good" if prob > 0.4 else "Poor" if prob > 0.05 else "Terrible"
        print(f"   {prob:4.2f}    ->   {log_val:6.2f}   ->   {loss_val:6.2f}  ({quality})")
    
    print(f"\nKey insight: Higher probability -> Lower loss (what we want!)")
    print(f"            Lower probability -> Higher loss (penalty for bad predictions)")
    
    return loss

def demonstrate_loss_calculation():
    print("\n" + "="*70)
    print("APPLYING LOSS TO OUR SEQUENCE: 'we love deep learning'")
    print("="*70)
    
    print("Now let's calculate loss for each word in our target sequence:")
    print("Formula: Loss = -log(P(correct_word))")
    print()
    
    words = ["we", "love", "deep", "learning"]
    total_loss = 0
    
    for i, (word, prob) in enumerate(zip(words, step_probabilities)):
        print(f"STEP {i+1}: Predicting '{word}'")
        print("-" * 25)
        print(f"  Model prediction: P({word}) = {prob:.3f}")
        print(f"  Take logarithm:   log({prob:.3f}) = {math.log(prob):.4f}")
        
        loss = -math.log(prob)
        total_loss += loss
        
        print(f"  Apply negative:   Loss = -({math.log(prob):.4f}) = {loss:.4f}")
        
        # Interpret the loss
        if loss < 0.5:
            interpretation = "Excellent prediction!"
        elif loss < 1.0:
            interpretation = "Good prediction"
        elif loss < 2.0:
            interpretation = "Decent prediction"
        else:
            interpretation = "Poor prediction - needs improvement"
            
        print(f"  Interpretation:   {interpretation}")
        print()
    
    print(f"TOTAL SEQUENCE LOSS:")
    print("-" * 20)
    print(f"Sum of all losses = {total_loss:.4f}")
    print(f"Average per word  = {total_loss/len(words):.4f}")
    print(f"\nTraining goal: Reduce this total loss by improving individual word predictions!")
    
    return total_loss

# First demonstrate the formula with toy example
toy_loss = demonstrate_cross_entropy_formula()

# Then apply to our actual sequence
sequence_loss = demonstrate_loss_calculation()

CROSS-ENTROPY LOSS FORMULA: Step-by-Step Breakdown
The cross-entropy loss formula is: Loss = -log(P(correct_word))
Let's break this down with a concrete example:

TOY EXAMPLE: Predicting 'cat' from vocabulary ['dog', 'cat', 'bird']
-----------------------------------------------------------------
Model predictions (probabilities):
  P(dog) = 0.2
  P(cat) = 0.6 <- CORRECT ANSWER
  P(bird) = 0.2

Verification: 1.0 (probabilities sum to 1.0)

STEP-BY-STEP LOSS CALCULATION:
-----------------------------------
Step 1: Identify the correct word
        Correct word = 'cat'

Step 2: Find its probability
        P(cat) = 0.6

Step 3: Take the natural logarithm
        log(P(cat)) = log(0.6) = -0.5108

Step 4: Apply the negative sign
        Loss = -log(P(cat)) = -(-0.5108) = 0.5108

FINAL RESULT: Loss = 0.5108

WHY NEGATIVE LOGARITHM?
Probability -> log(prob) -> -log(prob) (Loss)
---------------------------------------------
   0.90    ->    -0.11   ->     0.11  (Excellent)
   0.50    ->    -0